In [1]:
#imports
import pandas as pd
import numpy as np
from collections import Counter
import re
import csv
import seaborn as sns
import matplotlib.pylab as plt

import warnings
warnings.filterwarnings('ignore')
import os

In [2]:
C_PATH = '../Data/country/'
countries = os.listdir(C_PATH)
print(countries)

COUNTRY_LIST = []
for c in countries:
    c_code = c[:2]
    COUNTRY_LIST.append(c_code)
print(len(COUNTRY_LIST))

['ke.csv', 'gh.csv', 'in.csv', 'gb.csv', 'bd.csv', 'us.csv', 'jm.csv', 'tz.csv', 'ng.csv', 'hk.csv', 'pk.csv', 'au.csv', 'lk.csv', 'ca.csv', 'za.csv', 'sg.csv', 'ph.csv', 'ie.csv', 'my.csv', 'nz.csv']
20


## Topic assignments to LDA results

LDA model produced and assigned each documents to one of its class. Since after exhaustive searches and trials we decided up using 7 class and pruning percentage of 95.5 each documents are assigned a topic value from 0 u to 6.

Topic list is designed by us based on preliminary results (check all-pre-results folder). 

In [3]:
TOPIC_LIST = ['ENVIRONMENT/ENERGY','INTERNATIONAL','POLITICS','EDUCATION/FAMILY', \
              'SPORTS', 'TECHNOLOGY/SCIENCE/SOCIAL MEDIA', 'SOCIAL_LIFE/DAILY', \
              'ENTERTAINMENT/ART/MAGAZINE', 'COMPANY/BUSINESS', 'ECONOMY', \
              'POLICE/ACCIDENT/VIOLENCE', 'LEGAL/LAW','HEALTH/MEDICAL']
for ind, val in enumerate(TOPIC_LIST):
    print(ind, val)

0 ENVIRONMENT/ENERGY
1 INTERNATIONAL
2 POLITICS
3 EDUCATION/FAMILY
4 SPORTS
5 TECHNOLOGY/SCIENCE/SOCIAL MEDIA
6 SOCIAL_LIFE/DAILY
7 ENTERTAINMENT/ART/MAGAZINE
8 COMPANY/BUSINESS
9 ECONOMY
10 POLICE/ACCIDENT/VIOLENCE
11 LEGAL/LAW
12 HEALTH/MEDICAL


This dictionary is designed according to LDA-class -> Topic list match.
Structure is as following;

country_code : {topic_assigned_by_LDA: {topic_id_from_Topiclist: percentage} }

Here percentage enables us to assign multiple topic classes to a cluster discovered by LDA. For example LDA may discover topic_0 and news belonging to both POLITICS and INTERNATIONAL. With percentage approach, we are able to say 40\% of topic_0 accounts for POLITICS whereas 60\% accounts for INTERNATIONAL.    

In [4]:
 #country_code : {topic_assigned_by_LDA: {topic_id_from_Topiclist: percentage} }
COUNTRY_DICT = {}

COUNTRY_DICT['tz'] = {0:{6:90,12:10}, 1:{0:100}, 2:{1:50,2:50}, 3:{4:100}, 4:{5:100}, 5:{11:100}, 6:{8:50,12:50}} 
COUNTRY_DICT['bd'] = {0:{10:100}, 1:{1:50, 2:50}, 2:{1:100}, 3:{0:100}, 4:{6:100}, 5:{5:100}, 6:{11:100}}
COUNTRY_DICT['gh'] = {0:{0:50,12:50}, 1:{1:40,4:60}, 2:{8:50, 9:50}, 3:{4:30,7:70}, 4:{11:100}, 5:{2:100}, 6:{7:100}} 
COUNTRY_DICT['hk'] = {0:{5:30,8:70}, 1:{2:100}, 2:{6:100}, 3:{1:100}, 4:{8:50,9:50}, 5:{10:100}, 6:{7:100}}

COUNTRY_DICT['sg'] = {0:{7:100}, 1:{6:100}, 2:{10:100}, 3:{5:20,6:60,9:20}, 4:{8:100}, 5:{0:20,6:50,12:30}, 6:{5:100}} 
COUNTRY_DICT['pk'] = {0:{6:100}, 1:{2:100}, 2:{1:50,2:50}, 3:{1:100}, 4:{9:100}, 5:{2:50,1:50}, 6:{4:50,5:20,7:30}} 
COUNTRY_DICT['lk'] = {0:{0:100}, 1:{8:100}, 2:{2:40,10:60}, 3:{1:100}, 4:{5:100}, 5:{4:100}, 6:{6:100}} 
COUNTRY_DICT['my'] = {0:{5:100}, 1:{11:100}, 2:{2:90,10:10}, 3:{9:100}, 4:{0:100}, 5:{6:100}, 6:{5:60,8:40}} 

COUNTRY_DICT['nz'] = {0:{7:100}, 1:{8:10,11:90}, 2:{0:100}, 3:{4:100}, 4:{10:20,11:80}, 5:{12:100}, 6:{6:100}} 
COUNTRY_DICT['au'] = {0:{7:100}, 1:{4:100}, 2:{1:20,11:80}, 3:{12:100}, 4:{6:100}, 5:{9:100}, 6:{5:100}} 
COUNTRY_DICT['ca'] = {0:{12:100}, 1:{6:50,7:50}, 2:{10:100}, 3:{6:50,8:50}, 4:{0:100}, 5:{9:100}, 6:{0:30,4:70}} 
COUNTRY_DICT['gb'] = {0:{1:20,12:80}, 1:{4:50,6:50}, 2:{1:100}, 3:{7:10,11:90}, 4:{4:100}, 5:{7:100}, 6:{8:80,9:20}} 

COUNTRY_DICT['ng'] = {0:{4:100}, 1:{0:100}, 2:{1:50,2:50}, 3:{10:80,11:20}, 4:{7:100}, 5:{6:70,7:30}, 6:{9:100}} 
COUNTRY_DICT['za'] = {0:{1:50,2:50}, 1:{0:10,6:90}, 2:{1:50,12:50}, 3:{2:10,11:90}, 4:{5:50,12:50}, 5:{4:100}, 6:{9:100}} 

COUNTRY_DICT['ie'] = {0:{4:100}, 1:{11:100}, 2:{9:100}, 3:{6:50,12:50}, 4:{2:40,4:60}, 5:{2:100}, 6:{7:100}}
COUNTRY_DICT['in'] = {0:{2:100}, 1:{8:80,11:20}, 2:{9:100}, 3:{7:100}, 4:{1:20,4:80}, 5:{5:100}, 6:{2:100}}
COUNTRY_DICT['jm'] = {0:{0:20,12:80}, 1:{2:30,10:70}, 2:{8:60,12:40}, 3:{10:50,11:50}, 4:{6:100}, 5:{4:100}, 6:{7:100}} 
COUNTRY_DICT['ke'] = {0:{8:100}, 1:{8:50,9:50}, 2:{1:50,4:50}, 3:{6:100}, 4:{4:10,5:70,6:10,10:10}, 5:{6:20, 7:80}, 6:{10:50,11:50}}

COUNTRY_DICT['ph'] = {0:{2:100}, 1:{12:50}, 2:{0:100}, 3:{10:100}, 4:{9:100}, 5:{4:100}, 6:{6:100}}
COUNTRY_DICT['us'] = {0:{2:100}, 1:{2:40,10:60}, 2:{0:20,12:80}, 3:{6:70,11:30}, 4:{4:100}, 5:{8:50,9:50}, 6:{5:60,7:40}}

## Read source data and asignment data merge

To be able to get time, url etc features of articles we had to merge source information database already provided with article-topic dataframe we created. 

In [5]:
# Read news source folder as a whole
source_data = pd.read_table('../Data/now_sources_full.txt',encoding = "ISO-8859-1", header=None)
source_data.rename(columns={0:'textID', 1:'#words',2:'date',3:'country',4:'website',5:'url',6:'title'},inplace=True)
print('Total number of articles we have \(6 years\)) is ',len(source_data))
source_data.dropna(inplace=True)
source_data.date =  '20'+source_data.date
source_data.date =  pd.to_datetime(source_data.date, format='%Y-%m-%d')

Total number of articles we have \(6 years\)) is  6132175


In [6]:
# Divide docs belong to a cluster of LDA into multiple topic according to percentage
# NOTE: when we apply percentage distribution we did not sample the data, rather re-assigned topics in order

def assign_percentage_topics(topic_frame, topic_dist):
    """
    topic_frame: dataframe consists of doc_id and cluster_id(same for each)
    topic_dist: dictionary consists of topic-matches as we decided
    return: doc_id, topic_name dataframe
    """
    slices = []
    row_start = 0
    row_end = 0
    percen = 0
    for t_id, t_dis in topic_dist.items():
        
        percen = (percen+t_dis)/100
        row_end = row_start+  int(len(topic_frame)*percen)
        mini_slice = topic_frame[row_start:row_end]
        mini_slice.Topics = TOPIC_LIST[t_id]
        
        row_start = row_end
        slices.append(mini_slice)
    
    return pd.concat(slices) 


In [7]:
def get_article_by_country(c_code, c_dict):
    """
    c_code: two character country code
    c_dict: topic ssignments for that country
    """
    assignment_data = pd.read_csv(C_PATH+ c_code +'.csv', header=None)
    assignment_data.rename(columns={0:'textID', 1:'Topics'},inplace=True)
    topic_frames = []
    xx = []
    for assign_topic in c_dict.keys():
        t_fr = assign_percentage_topics(assignment_data[assignment_data.Topics==assign_topic],\
                                        c_dict[assign_topic])
        topic_frames.append(t_fr)
    return pd.concat(topic_frames),xx

In [8]:
c_frames = []
for cind in range(len(COUNTRY_LIST)) :
    c_code = COUNTRY_LIST[cind]
    c_dict = COUNTRY_DICT[c_code]
    per_country,xx = get_article_by_country(c_code, c_dict)
    c_frames.append(per_country)

all_counries = pd.concat(c_frames)

In [9]:
# All necessary infomations to run correlations
full_data = pd.merge(source_data, all_counries, left_on='textID', right_on='textID')
# News_Id, Topic of the News and all others...
full_data.head()

,textID,#words,date,country,website,url,title,Topics
0,3732490,815,2015-11-01,US,NPR,http://www.npr.org/2015/11/01/450889721/the-ma...,The Madonna Of 115th Street Gets A Long-Awaite...,SOCIAL_LIFE/DAILY
1,3732492,258,2015-11-01,US,Huffington Post,http://www.huffingtonpost.com/entry/university...,University Of Louisville Sorry A Bunch Of Its ...,SOCIAL_LIFE/DAILY
2,3732496,489,2015-11-01,US,Bleacher Report,http://bleacherreport.com/articles/2584708-kar...,'Kareem: Minority of One' HBO Documentary Prev...,POLITICS
3,3732501,840,2015-11-01,US,VentureBeat,http://venturebeat.com/2015/11/01/what-big-ind...,What big industry will do to the Internet of T...,HEALTH/MEDICAL
4,3732502,470,2015-11-01,US,Tech Insider,http://www.techinsider.io/daenerys-game-of-thr...,Daenerys has been traveling in the wrong direc...,SPORTS


In [10]:
# Total number of news have a specific topic per country
g_country_topic = full_data.groupby(by=['country','Topics'])['textID'].count()
g_country = g_country_topic.groupby(by=['country']).sum()

In [11]:
# Percentages of topics
g_percentage = g_country_topic/g_country
g_percentage

country  Topics                         
AU       ECONOMY                            0.157716
         ENTERTAINMENT/ART/MAGAZINE         0.153065
         HEALTH/MEDICAL                     0.141965
         INTERNATIONAL                      0.029435
         LEGAL/LAW                          0.117698
         SOCIAL_LIFE/DAILY                  0.112324
         SPORTS                             0.124271
         TECHNOLOGY/SCIENCE/SOCIAL MEDIA    0.163525
BD       ENVIRONMENT/ENERGY                 0.157019
         INTERNATIONAL                      0.175870
         LEGAL/LAW                          0.212020
         POLICE/ACCIDENT/VIOLENCE           0.175870
         POLITICS                           0.034819
         SOCIAL_LIFE/DAILY                  0.133289
         TECHNOLOGY/SCIENCE/SOCIAL MEDIA    0.111111
CA       COMPANY/BUSINESS                   0.073039
         ECONOMY                            0.138273
         ENTERTAINMENT/ART/MAGAZINE         0.073039
     

Following cells, we created a matrix shows for each country, percentage of the topic (non values if not match in that country for that topic). 

In [12]:
topics_distribution = pd.DataFrame(full_data['country'].unique()) 
topics_distribution.rename(columns={0:'country'},inplace=True)
topics_distribution['country_name'] = ['United States', 
                                       'Canada', 
                                       'United Kingdom',
                                       'Ireland',
                                        'Australia',
                                        'New Zealand',
                                        'India',
                                        'Sri Lanka',
                                       'Pakistan', 
                                       'Bangladesh',
                                        'Malaysia',
                                        'Singapore',
                                        'Philippines',
                                       'Hong Kong',
                                       'South Africa',
                                        'Nigeria', 
                                        'Ghana',
                                        'Kenya',                           
                                        'Tanzania',
                                       'Jamaica']

In [13]:
# Make a none matrix ready
for col in TOPIC_LIST:
    topics_distribution[col] = None

In [14]:
g_percentage_pd = pd.DataFrame(g_percentage)
g_percentage_pd.reset_index(inplace=True)
g_percentage_pd.head()

,country,Topics,textID
0,AU,ECONOMY,0.157716
1,AU,ENTERTAINMENT/ART/MAGAZINE,0.153065
2,AU,HEALTH/MEDICAL,0.141965
3,AU,INTERNATIONAL,0.029435
4,AU,LEGAL/LAW,0.117698


In [15]:
# Fill the matrix
for ind, row in g_percentage_pd.iterrows():
    country = row.country
    topic = row.Topics
    freq = row.textID
    topics_distribution.loc[topics_distribution['country'] == country, topic] = freq

In [16]:
# Topic distribution matrix
topics_distribution.head()

,country,country_name,ENVIRONMENT/ENERGY,INTERNATIONAL,POLITICS,EDUCATION/FAMILY,SPORTS,TECHNOLOGY/SCIENCE/SOCIAL MEDIA,SOCIAL_LIFE/DAILY,ENTERTAINMENT/ART/MAGAZINE,COMPANY/BUSINESS,ECONOMY,POLICE/ACCIDENT/VIOLENCE,LEGAL/LAW,HEALTH/MEDICAL
0,US,United States,0.022056,None,0.194978,None,0.113442,0.0828643,0.13251,0.0552429,0.0848248,0.084809,0.0845718,0.0566184,0.0880818
1,CA,Canada,0.194973,None,None,None,0.0958403,None,0.1466,0.0730386,0.0730386,0.138273,0.165221,None,0.113016
2,GB,United Kingdom,None,0.175466,None,None,0.22891,None,0.0572487,0.147708,0.148592,0.0372031,None,0.121004,0.0838684
3,IE,Ireland,None,None,0.177962,None,0.252041,None,0.0542922,0.184377,None,0.143317,None,0.13386,0.0541523
4,AU,Australia,None,0.0294349,None,None,0.124271,0.163525,0.112324,0.153065,None,0.157716,None,0.117698,0.141965


In [17]:
# Save
topics_distribution.to_csv('../Data/topic_distribution.csv', index=False)

In [ ]:
# Save
full_data.to_csv('../Data/final-news-data.csv', index=False)

## Most frequent words appeared in dominant topics for each country

After calculations and visualizations (plots notebook), dominant topic for each country per year is know. This are 5 most frequent words (out of 10) found by LDA regarding the dominant topic. These words are extracted manual examinations of LDA results for each country. One can reach full list of words per topic per country under LDAResults directory.

In [ ]:
# Dominant topics is chosen as the one which biggest particion of docs are assigned
words_c = { 'us':['electoral','romney','objectionable','pentagon','libertarian','POLITICS'],
         'ie':['klopp','sunderland','spur','coleman','horgan','SPORTS'],
         'ca':['fossil','tracking','hectare','brook','panther','ENVIRONMENT/ENERGY'],
         'gb':['wimbledon','milan','diego','clark','spearhead','SPORTS'],
         'gh':['entertainment','wear','actress','gospel','george','ENTERTAINMENT/ART/MAGAZINE'],
         'bd':['lawyer','sentence','supreme','petition','allegation','LEGAL/LAW'],
         'hk':['insurance','marketing','partneship','secure','monetary','COMPANY/BUSINESS'],
         'ng':['stock','trillion','regulatory','manufacturing','trader','ECONOMY'],
         'pk':['congress','erdogan','coup','china-pakistan','census','POLITICS'],
         'sg':['merger','deutsche','malay','civillian','religion','SOCIAL_LIFE/DAILY'],
         'tz':['operator','online','user','stream','provider','TECHNOLOGY/SCIENCE/SOCIAL MEDIA'],
         'za':['teammate','fitness','warrior','celtic','winning','SPORTS'],
         'in':['puja','shaka','azad','anti-nation','liberal','POLITICS'],
         'jm':['video','perform','dance','popular','singer','ENTERTAINMENT/ART/MAGAZINE'],
         'ke':['innovation','consumption','planning','profit','machinery','COMPANY/BUSINESS'],
         'lk':['disaster','submit','flood','coal','allege','ENVIRONMENT/ENERGY'],
         'my':['auto','circuit','researcher','tech','computer','TECHNOLOGY/SCIENCE/SOCIAL MEDIA'],
         'nz':['copyright','prohibit','prosecutor','supreme','suspend','LEGAL/LAW'],
         'ph':['firearm','supt','raid','calamity','isolated','POLICE/ACCIDENT/VIOLENCE'],
         'au':['nasa','integrate','wireless','hybrid','audio','TECHNOLOGY/SCIENCE/SOCIAL MEDIA']
}

In [ ]:
word_freq =  pd.DataFrame.from_dict(words_c)
word_freq

In [ ]:
word_freq.to_csv('../Data/most_freq_words.csv', index=False)